# try to slip

In [ ]:
import pickle
import pandas as pd
import transformers
from transformers import GPT2Tokenizer
import random
import re

def clean_text(text):
    text = re.sub(r'\n+', ' ', text)
    text = re.sub(r'\t+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text

# 初始化 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2", sep_token="[SEP]", pad_token="[PAD]", cls_token="[CLS]")
# tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium", sep_token="[SEP]", pad_token="[PAD]", cls_token="[CLS]")
sep_id = tokenizer.sep_token_id
pad_id = tokenizer.pad_token_id
cls_id = tokenizer.cls_token_id
special_tks = list()
special_tks.append('#@#') # code separator
special_tks.append('<NL>') # code separator
# special_tks.append('<None>') # code separator
special_tks.append('[CLS]') # [CLS] 開頭
special_tks.append('[PAD]') # [PAD] 中間
special_tks.append('[SEP]') # [SEP] 結束
tokenizer.add_special_tokens({"additional_special_tokens": special_tks})

max_length = 0
longest_data = None
data_set = []
data_set2 = []
# big_label_list = []

too_longdata = []
kmuh_reports_dict = dict()

data_filepath = r'C:\Users\Administrator\Desktop\old gpt2\aicup\data'
data_filepath = r'C:\Users\Administrator\Desktop\old gpt2\aicup\data_old'
data_filepath = r'C:\Users\Administrator\Desktop\old gpt2\aicup\data_old_new'
lable_filepath = r'C:\Users\Administrator\Desktop\old gpt2\aicup\answer.txt'

import os
import glob
from tqdm import tqdm

# 读取answer.txt中的标签数据
with open(lable_filepath, 'r', encoding='utf-8') as label_file:
    labels = label_file.readlines()

txt_files = glob.glob(os.path.join(data_filepath, '*.txt'))

# for txt_file in txt_files:
for txt_file in tqdm(txt_files, desc='Processing files', unit='file'):
    # print(txt_file)
    with open(txt_file, 'r', encoding='utf-8') as data_file:
        data = data_file.read()
    data = clean_text(data)

    slip_num = 15
    split_size = len(data) // slip_num #+ ((len(data) // (slip_num-1) - len(data) // slip_num)//int(slip_num*2.5))

    gc = 0

    splits = []
    start_index = 0

    while start_index < len(data):
        if gc == slip_num:
            print('warring1=', split_size)

        end_index = start_index + split_size

        # Check if the next character after the split is a space
        if end_index < len(data) and data[end_index] != ' ':
            # Move the end index forward until a space is found
            while end_index < len(data) and data[end_index] != ' ':
                end_index += 1

        splits.append(data[start_index:end_index])
        gc += 1
        start_index = end_index

    # if gc == (slip_num-2):
    #     print('warring2=', split_size)

        # splits = []
        # splits.append(data)
    label_list = []
    for label in labels:
        label_parts = label.split('\t')
        if label_parts[0] == os.path.basename(txt_file).split('.')[0]:
            label_list.append(label)
            # big_label_list.append(label)
    for data_slipt in splits:
        matched_label = ''
        data_with_label = ''
        data_with_label2 = ''
        for j in label_list:
            label_parts = j.split('\t')
            # print( label_parts[4] )
            if label_parts[4].strip() in data_slipt.strip():
                try:
                    matched_label += label_parts[1].strip() + "\t" + label_parts[4].strip() + "\t" + label_parts[5].strip() + '#@#'
                except:
                    matched_label += label_parts[1].strip() + "\t" + label_parts[4].strip() + '#@#'
        matched_label = re.sub(r'\t', '<T>', matched_label)
        matched_label = re.sub(r'\s+', '<NL>', matched_label)
        matched_label = re.sub('<T>', r'\t', matched_label)
        data_slipt = re.sub(r'\t', '<T>', data_slipt.strip())
        data_slipt = re.sub(r'\s+', '<NL>', data_slipt)
        data_slipt = re.sub('<T>', r'\t', data_slipt)
        data_with_label = data_slipt + tokenizer.sep_token + matched_label.replace('\n','').rstrip('#@#')
        data_with_label2 = data_slipt + " => " + matched_label.replace('\n','').rstrip('#@#')


    #     if len(data_with_label) > 1024:
            # print(txt_file)
    #         continue
        # print('data_with_label=\n', data_with_label)
        data_set.append(data_with_label)
        data_set2.append(data_with_label2)

data_set = data_set + data_set + data_set + data_set + data_set + data_set
data_set = data_set + data_set
data_with_label3 = data_set

train_list = list() # 转换为列表
max_length = 0
for data in data_set:
    if len(data.split("<NL>")) > max_length:
        max_length = len(data.split("<NL>"))
    input_ids = [cls_id] # 以 [CLS] 開頭
    input_ids += tokenizer.encode(data, add_special_tokens=False)
    input_ids.append(sep_id) # 加上 [SEP], 表示報告內容結束
    train_list.append(input_ids)
    
print("最長的數量是:", max_length)

import random
# 隨機排序資料集
random.shuffle(train_list)
# 計算切分的索引
train_size = int(0.98 * len(train_list))
# 切分訓練集和測試集
train_set = train_list[:train_size]
test_set = train_list[train_size:]

# 保存训练集到train.pkl
with open(r'C:\Users\Administrator\Desktop\old gpt2\aicup\dataset\train.pkl', "wb") as f:
    pickle.dump(train_set, f)

# 保存测试集到test.pkl
with open(r'C:\Users\Administrator\Desktop\old gpt2\aicup\dataset\test.pkl', "wb") as f:
    pickle.dump(test_set, f)
train_list = list() # 转换为列表

# 写入数据到文件
with open(r'C:\Users\Administrator\Desktop\old gpt2\aicup\dataset\train.txt', "w", encoding='utf-8') as file:
    for j in data_with_label3:
        file.write(f"{j}\n")

print("訓練集数量:", len(train_set))
print("測試集数量:", len(test_set))


## vaild - pkl

In [ ]:
import pickle
import pandas as pd
import transformers
from transformers import GPT2Tokenizer
import random
import re

def clean_text(text):
    text = re.sub(r'\n+', ' ', text)
    text = re.sub(r'\t+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text

# 初始化 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2", sep_token="[SEP]", pad_token="[PAD]", cls_token="[CLS]")
# tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium", sep_token="[SEP]", pad_token="[PAD]", cls_token="[CLS]")
sep_id = tokenizer.sep_token_id
pad_id = tokenizer.pad_token_id
cls_id = tokenizer.cls_token_id
special_tks = list()
special_tks.append('#@#') # code separator
special_tks.append('<NL>') # code separator
# special_tks.append('<None>') # code separator
special_tks.append('[CLS]') # [CLS] 開頭
special_tks.append('[PAD]') # [PAD] 中間
special_tks.append('[SEP]') # [SEP] 結束
tokenizer.add_special_tokens({"additional_special_tokens": special_tks})

max_length = 0
longest_data = None
data_set = []
data_set2 = []

too_longdata = []
kmuh_reports_dict = dict()

data_filepath = r'C:\Users\Administrator\Desktop\old gpt2\aicup\Second_Phase_Dataset\Second_Phase_Dataset\Second_Phase_Text_Dataset'
lable_filepath = r'C:\Users\Administrator\Desktop\old gpt2\aicup\Second_Phase_Dataset\Second_Phase_Dataset\answer.txt'
data_filepath = r'C:\Users\Administrator\Desktop\old gpt2\aicup\First_Phase_ReleaseCorrection\First_Phase_Release(Correction)\Validation_Release'
lable_filepath = r'C:\Users\Administrator\Desktop\old gpt2\aicup\First_Phase_ReleaseCorrection\answer.txt'

import os
import glob
from tqdm import tqdm

# 读取answer.txt中的标签数据
with open(lable_filepath, 'r', encoding='utf-8') as label_file:
    labels = label_file.readlines()

txt_files = glob.glob(os.path.join(data_filepath, '*.txt'))

for txt_file in tqdm(txt_files, desc='Processing files', unit='file'):
    with open(txt_file, 'r', encoding='utf-8') as data_file:
        data = data_file.read()
    data = clean_text(data)

    flag = False
    flag2 = False

    slip_num = 15
    split_size = len(data) // slip_num #(slip_num + ((len(data) // (slip_num-2) - len(data) // (slip_num-1) )//int(slip_num*2.5)))
    gc = 0

    splits = []
    start_index = 0
    
    while start_index < len(data):
        if gc == slip_num:
            print('warring1=', split_size)
        
        end_index = start_index + split_size

        # Check if the next character after the split is a space
        if end_index < len(data) and data[end_index] != ' ':
            # Move the end index forward until a space is found
            while end_index < len(data) and data[end_index] != ' ':
                end_index += 1

        splits.append(data[start_index:end_index])
        gc += 1

        start_index = end_index
    if gc == (slip_num-2):
        print('warring2=', split_size)
    # if split_size > 400:
    #     print('a>400',split_size)
    if len(splits) != slip_num:
        print('some error', len(splits), " split_size=",split_size,)
     
    for data_slipt in splits:
        data_with_label2 = ''
        data_slipt = re.sub(r'\t', '<T>', data_slipt.strip())
        data_slipt = re.sub(r'\s+', '<NL>', data_slipt)
        # data_slipt = re.sub('<NL> <NL>', '<NL>', data_slipt)
        data_slipt = re.sub('<T>', r'\t', data_slipt)
        # data_slipt = clean_text(data_slipt)
        data_with_label2 = data_slipt
        data_set2.append(data_with_label2)
train_list = list() # 转换为列表

max_length = 0
for data in data_set2:
    if len(data.split("<NL>")) > max_length:
        max_length = len(data.split("<NL>"))

    input_ids = [cls_id] # 以 [CLS] 開頭
    input_ids += tokenizer.encode(data, add_special_tokens=False)
    input_ids.append(sep_id) # 加上 [SEP], 表示報告內容結束
    train_list.append(input_ids)
print("最長的數量是:", max_length)

# 保存测试集到test.pkl
with open(r'C:\Users\Administrator\Desktop\old gpt2\aicup\dataset\test2.pkl', "wb") as f:
    pickle.dump(train_list, f)
print("訓練集数量:", len(train_list))

train_list = list() # 转换为列表

for data in data_set2:
    train_list.append(data)
with open(r'C:\Users\Administrator\Desktop\old gpt2\aicup\dataset\test.txt', "w", encoding='utf-8') as file:
    for j in train_list:
        file.write(f"{j}\n")

print("測試集数量:", len(train_list))


# find answer position

In [ ]:
file_path = r"C:\Users\Administrator\Desktop\old gpt2\aicup\answer.txt"
file_path = r"C:\Users\Administrator\Desktop\old gpt2\aicup\First_Phase_ReleaseCorrection\answer.txt"
element_dict = {}

with open(file_path, "r", encoding="utf-8") as file:
    for line in file:
        elements = line.strip().split("\t")
        if len(elements) >= 2:
            element = elements[1]
            element_dict[element] = True

element_list = list(element_dict.keys())
print(element_list)

In [ ]:
import pickle
import pandas as pd
import transformers
from transformers import GPT2Tokenizer
from tqdm import tqdm
import re

def clean_text(text):
    # text = re.sub(',', ' ', text)
    # text = re.sub(r'\t+', '\t', text)
    text = re.sub(r'\s', ' ', text)
    return text

max_length = 0
longest_data = None
data_set = []
data_set2 = []

too_longdata = []
kmuh_reports_dict = dict()

data_filepath = r'C:\Users\Administrator\Desktop\old gpt2\aicup\Second_Phase_Dataset\Second_Phase_Dataset\Second_Phase_Text_Dataset'
data_filepath = r'C:\Users\Administrator\Desktop\old gpt2\aicup\First_Phase_ReleaseCorrection\First_Phase_Release(Correction)\Validation_Release'
lable_filepath = r'C:\Users\Administrator\Desktop\old gpt2\aicup\dataset\predict.txt'

import os
import glob


# 读取answer.txt中的标签数据
with open(lable_filepath, 'r', encoding='utf-8') as label_file:
    labels = label_file.readlines()
# 移除空行
labels = [label for label in labels if label.strip()]

txt_files = glob.glob(os.path.join(data_filepath, '*.txt'))

for txt_file in tqdm(txt_files, desc='Processing files', unit='file'):
    # print(txt_file)
    with open(txt_file, 'r', encoding='utf-8') as data_file:
        data = data_file.read()
    data = clean_text(data)
    txt_file = txt_file.replace('C:\\Users\\Administrator\\Desktop\\old gpt2\\aicup\\First_Phase_ReleaseCorrection\\First_Phase_Release(Correction)\\Validation_Release\\','').replace('.txt', '')
    # print(txt_file)
    label_count = {}
    # label_temporary = []
    for label in labels:
        word = ''
        if label.strip() == '=====================':
            break
        try:
            label_parts = label.split('\t')
            word = label_parts[1].strip()
            word = str(word) #.strip())
        except:
            continue

        try:
            label_count[word] += 1
        except:
            label_count[word] = 0
    for label in labels:
        # print(label)
        matches = []
        # start_index = 0
        # end_index = 0
        word = ''
        if label.strip() == '=====================':
            break
        try:
            label_parts = label.split('\t')
            word = label_parts[1].strip()
            word = str(word) #.strip())
        except:
            continue
        try:
            pattern = (word) + r"\b"
            matches = list(re.finditer(pattern, str(data)))
        except:
            continue

        if not matches:
            continue

        falg = False

        if label_count[word] == 0:
            start_index = matches[int(label_count[word])].start()
            end_index = matches[int(label_count[word])].end()
        while label_count[word] > 0:
            try:
                start_index = matches[label_count[word]].start()
                end_index = matches[label_count[word]].end()
                break  # 如果成功取得索引，跳出迴圈
            except:
                pass
            finally:
                label_count[word] = label_count[word] - 1

        label_parts = label.split('\t', 1)
        show = txt_file + '\t' + label_parts[0] + '\t' + str(start_index) + '\t' + str(end_index) + '\t' + label_parts[1] 
        data_set.append(show)

    try:
        break_line = '====================='
        break_index = None
        # 寻找第一个"=="所在行的索引
        for i, label in enumerate(labels):
            if label.strip() == break_line:
                break_index = i
                break
        if break_index is not None:
            # 移除已执行过的内容
            labels = labels[break_index + 1:]
    except:
        print("finall = ", txt_file)
        continue
with open(r'C:\Users\Administrator\Desktop\old gpt2\aicup\dataset\answer.txt', "w", encoding='utf-8') as file:
    for j in data_set:
        file.write(f"{j}")
